<h4>
Resolução da atividade prática:
</h4>

<p>
Tópico A - Implemente o algoritmo K-Means visto em sala de aula<br>
    <br>i.	Medida de proximidade: distância Euclidiana
    <br>ii.	Centróides iniciais: escolher aleatoriamente K objetos da base de dados
    <br>iii.	K: parâmetro de entrada, especificado pelo usuário
    <br>iv.	O laço de repetição executado até a convergência do algoritmo terá limite máximo de 100 iterações
    <br>v.	Casos de empate na associação de um elemento ao centróide: escolher o primeiro.
</p>

<p>
Tópico B - Execute a técnica implementada usando uma base de dados pública, como por exemplo a base Íris<br>
</p>

In [36]:
from sklearn.preprocessing import MinMaxScaler
from tkinter import filedialog
import tkinter as tk
import pandas as pd
import numpy as np
import math

In [37]:
def select_file():

    root = tk.Tk()
    root.withdraw() 
    arquivo = filedialog.askopenfilename(title="Selecione um arquivo .csv", 
                                         filetypes=(("Arquivos CSV", "*.csv"), 
                                         ("Todos os arquivos", "*.*")))
    return arquivo 

def dropClass(column, dataset):
    dataset.drop(column, axis=1, inplace=True)
    return dataset

def normalize(dataset):
    return (dataset-dataset.min())/(dataset.max()-dataset.min())

In [38]:
'''
    Importando o dataset a ser usado no
    processo de agrupamento.
    
    * A função select_file() utiliza a  
      biblioteca tkinter é utilizada
      para criar uma janela de diálogo 
      de seleção de arquivo

    * A função dropClass(column, dataset)
      exclui alguma coluna, a ser especificado
      pelo nome no parâmetro 'column'

    * A função normalize(dataset) aplica
      a normalização min-max no dataset
'''

dataframe = select_file()
datasetPD = pd.read_csv(dataframe)

classColumn = input("Nome da coluna de classes a ser excluída")
if classColumn in datasetPD:
    datasetPD = dropClass(classColumn, datasetPD)
else:
    datasetPD = datasetPD
display(datasetPD)

,A,B,C,D
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [41]:
'''
    Processo de normalização
    * Inicialmente é feita uma validação se há
      dados não numericos no dataset. Neste caso
      não é feita a normalização.
'''
count = 0
for i in range(0,datasetPD.shape[1]):
    if datasetPD[datasetPD.columns[i]].dtype == 'float64' or datasetPD[datasetPD.columns[i]].dtype == 'int64':
        count += 1

if count == datasetPD.shape[1]:
    scaler = MinMaxScaler()
    datasetNorm = scaler.fit_transform(datasetPD)
    
    # Exibindo os 5 primeiros elementos do dataset
    # a fim de verificar a aplicação da normalização
    display(datasetNorm[:5], type(datasetNorm)) 

else:
    print('A base de dados contém atributos não numerico, não foi possível normalizar')


array([[0.22222222, 0.625     , 0.06779661, 0.04166667],
       [0.16666667, 0.41666667, 0.06779661, 0.04166667],
       [0.11111111, 0.5       , 0.05084746, 0.04166667],
       [0.08333333, 0.45833333, 0.08474576, 0.04166667],
       [0.19444444, 0.66666667, 0.06779661, 0.04166667]])

numpy.ndarray

In [80]:
''' 
    Criação da classe K_means, sendo dotado das seguintes funções:

    * __init__ -> Caso os parâmetros não sejam definidos no momento da instância
                  valores padrões são utilizados para a inicialização, sendo eles:
                  - k = quantidade desejada de clusters
                  - max_iterations = valor máximo de iterações do algoritmo 
                  - tolerance = tolerância de variação dos centróides

    * euclidean_distance -> Realiza o calculo da distância eclidiana entre dois 
                            pontos definidos 
                        
    * predict -> Realiza a predição de um cluster para um dado objeto
    * fit -> Função de treinamento. É responsável por executar o kmeans
      e definir os cluster, bem como incluir os objetos a cada cluster
'''

class K_Means:

    def __init__(self, k = 2, tolerance = 0.001, max_iter = 100):
        self.k = k
        self.max_iterations = max_iter
        self.tolerance = tolerance
    
    def euclidean_distance(self, point1, point2):
        return math.sqrt((point1[0]-point2[0])**2 + (point1[1]-point2[1])**2) 
        
    def predict(self, point): 
        '''  
            É realizado o calculo das distâncias entre um determinado ponto
            e cada um dos centróides definidos pelo agrupamento. O cluster que
            resultar na menor distância sera o resultado da predição

            obs: A função np.linalg.norm() retorna a distância euclidiana,
                 o uso desta função é mais eficiente para este calculo  
        '''
        distances = [np.linalg.norm(point - self.centroids[centroid]) for centroid in self.centroids]
        classification = distances.index(min(distances))
        return classification
    

    def fit(self, data):

        '''  
            A inicialização dos centróides é feita de maneira simples:
            * São escolhidos K centróides aleatoriamente do dataset.
        '''
        np.random.seed(3)
        self.centroids = {} # Inicialização do dicionário de centroides

        for i in range(self.k):
            self.centroids[i] = data[np.random.choice(len(data), 1, replace=False)[0]]
        
        
        for i in range(self.max_iterations): 

            self.classes = {} # Inicialização do dicionário de classes

            for j in range(self.k):
                self.classes[j] = [] # Inicialização do array de distâncias
                
            '''  
                Sera computada as distancias euclidianas de um determinado 
                ponto i com todos os centróides definidos. Após o preenchimento
                da lista de distâncias, será definido o cluster_index, que 
                representa o índice do cluster que obteve a menor distância 
                euclidiana com o ponto (O mesmo cluster_index também será 
                utilizado como index da 'classe' para definir para qual cluster
                o ponto será alocado).  

                Obs: O mesmo processo será repetido para todos os pontos do dataset
   
            '''
            for point in data:
                distances = []
                for index in self.centroids:
                    distances.append(self.euclidean_distance(point, self.centroids[index]))
                cluster_index = distances.index(min(distances))
                self.classes[cluster_index].append(point)

            '''  
                Previous é um dicionário python com base nos centróides, que 
                armazena os valores prévios dos centróides definidos
            '''
            previous = self.centroids
        
            ''' 
                Para cada índice de cluster dentro do dicionário de classes
                é calculado um novo valor de centróide, levando em consideraço a média 
                dos valores dos pontos que foram alocados do dicionário 'classes' no 
                trecho de código anterior: self.classes[cluster_index].append(point)

                obs: A flag isOptimal é definida como True
            '''
            for cluster_index in self.classes:
                self.centroids[cluster_index] = np.average(self.classes[cluster_index], axis = 0) 
            isOptimal = True
            
            '''  
                Para cada centroid definido, será avaliada o qual foi a variação
                de valor do novo centróide calculado. Se a porcentagem da varição
                for superior à tolerância estabelecida previamente, o loop de execução
                continua, caso contrário (isOptimal is True), a execução é encerrada. 
            '''
            for centroid in self.centroids:
                original_centroid = previous[centroid]
                curr = self.centroids[centroid]
                if np.sum((curr - original_centroid)/original_centroid * 100.0) > self.tolerance:
                    isOptimal = False
            if isOptimal:
                break

In [81]:
'''  
    Definição da instância da classe k_means criada
    e treinamento com a base de dados iris_dataset
'''
k_means = K_Means(k = 3, tolerance = 0.001, max_iter = 100)
k_means.fit(datasetNorm)

<P>
Tópico C - O programa deverá produzir como saída um arquivo indicando para cada elemento, o grupo ao qual ele foi associado.<br>   
Tópico D - Se estiver usando alguma ferramenta de alto nível para programa em Python, é necessario de enviar também o arquivo .py  
</P>

In [82]:
''' 
    Os dados de predição serão adicionados 
    ao dataset original (como uma coluna extra
    indicado o cluster ao qual o objeto pertence)
    e o novo conteúdo será exportado em formato CSV.
'''
classes = []
for i in range(len(datasetNorm)):
    classes.append(k_means.predict(datasetNorm[i]))

OutputCSV = pd.read_csv(dataframe)

if 'Grupo' not in OutputCSV:
    OutputCSV.insert(OutputCSV.shape[1], 'Grupo', classes, False)

else:
    print('Classificação já atribuída')

OutputCSV.to_csv('Resultado.csv' )
display(OutputCSV)

,A,B,C,D,ESPECIE,Grupo
0,5.1,3.5,1.4,0.2,Iris-setosa,0
1,4.9,3.0,1.4,0.2,Iris-setosa,0
2,4.7,3.2,1.3,0.2,Iris-setosa,0
3,4.6,3.1,1.5,0.2,Iris-setosa,0
4,5.0,3.6,1.4,0.2,Iris-setosa,0
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica,2
146,6.3,2.5,5.0,1.9,Iris-virginica,2
147,6.5,3.0,5.2,2.0,Iris-virginica,2
148,6.2,3.4,5.4,2.3,Iris-virginica,2
